# Calculador de Sazonalidade Mensal da Receita Prevista

Este Jupyter Notebook calcula a sazonalidade mensal de determinada receita de acordo com as arrecadações mensais em anos anteriores.

## Procedimentos de cálculo

Data uma determinada receita, determinado valor anual previsto e determinado intervalo de anos anteriores:

1. Para cada ano anterior, identifica o valor mensal arrecadado e o total arrecadado no ano;
2. Para cada ano anterior, calcula o peso percentual da arrecadação mensal em relação ao total arrecadado;
3. Para cada mês, apura a média do peso percentual arrecadado;
4. Para cada mês, podera os pesos mensais de forma que a soma de pesos mensais seja igual a 100%;
5. Tendo por base o valor anual previsto, para cada mês aplica o percentual médio ponderado a fim de obter a previsão mensal da receita.

## Especificidades do cálculo

### Garantia de que a soma dos meses seja igual à receita total

Para evitar que a soma dos valores mensais seja diferente do valor total em virtude de arredondamento, tanto os percentuais quanto so valores finais relativos ao mês de dezembro é dado subtraindo-se a soma de janeiro a novembro do total.

### Arredondamento dos valores mensais previstos

Foi adotado um coeficiente de arredondamento para ser usado nos valores previstos para janeiro a novembro, sendo a diferença acumulada para o mês de dezembro.

## Origem dos dados

Os dados utilizados são os do arquivo RECEITA.TXT gerado para o SIAPC/PAD do TCE/RS convertidos pelo [pad_converter](https://github.com/iddrs/pad-converterhttps://github.com/iddrs/pad-converter).

In [44]:
# Configurando o ambiente

import pandas as pd
from os import path

'''
Diretório base para os dados.

Dentro desse diretório deve haver um subdiretório para cada ano especificado em ```anos_anteriores``` (definida no bloco seguinte)

Dentro de cada subdiretório é necessário constar um arquivo *RECEITA.CSV*.
'''
dir_base_dados = r'./dados'


In [2]:
# Configuração de/para dos códigos da receita

'''
Graças às mudanças mais frequentes do que gostaríamos na codificação da receita (obrigado STN, obrigado TCE/RS), é necessário fazer um de/para nos códigos das receitas.

A estrutura do de/para é a seguinte:

- Dicionário onde chave = identificador da receita e valor = lista
    - Lista
        - String com um identificador (nome) da receita
        - Dicionário onde chave = ano e valor = código da receita naquele ano
'''

de_para = {
    'iptu_principal': ['IMPOSTO SOBRE A PROPRIEDADE PREDIAL E TERRITORIAL URBANA - PRINCIPAL',
    {
        2019: '4.1.1.1.8.01.1.1.00.00.00.00.000',
        2020: '4.1.1.1.8.01.1.1.00.00.00.00.000',
        2021: '4.1.1.1.8.01.1.1.00.00.00.00.000'
    }],
}

In [40]:
# Configurando as bases do cálculo

'''
Lista dos anos que serão considerados para o cálculo dos percentuais.

Obviamente que presia ser um intervalo constante de anos. Então, nada de ficar "pulando" anos por aí.

Além disso, para cada ano é necessário existir um diretório dentro de *dados* e dentro deles, um arquivo *RECEITA.CSV*
'''
anos_anteriores = (2019, 2020, 2021)

'''
O valor de ```ndigits``` da [função built-in](https://docs.python.org/3/library/functions.html#round) do Python ```round()```.
'''
coeficiente_arredondamento = -3

In [4]:
# Configurando o cálculo

# A chave da receita alvo usado em ```de_para```
receita_alvo = 'iptu_principal'

# Valor previsto para a receita alvo. É o valor que queremos dividir de acordo com a sazonalidade mensal
valor_receita_alvo = 596_000.00

In [10]:
# Executando o cálculo
# Não mude nada a partir daqui, exceto se souber o que estás fazendo

# Carrega os dados de receita

# Lista com as colunas a retornar no dataframe
colunas_nomes = [
    'codigo_receita',
    'receita_realizada_jan',
    'receita_realizada_fev',
    'receita_realizada_mar',
    'receita_realizada_abr',
    'receita_realizada_mai',
    'receita_realizada_jun',
    'receita_realizada_jul',
    'receita_realizada_ago',
    'receita_realizada_set',
    'receita_realizada_out',
    'receita_realizada_nov',
    'receita_realizada_dez'
]

# Armazena os dataframes com os dados das receitas
dados_receita = {}
for ano in anos_anteriores:
    f = path.join(dir_base_dados, str(ano), 'RECEITA.csv')
    print(f'Carregando dados de {ano} a partir de {f}...')
    df = pd.read_csv(f, sep=';', usecols=colunas_nomes, thousands='.', decimal=',')
    dados_receita[ano] = df

Carregando dados de 2019 a partir de ./dados\2019\RECEITA.csv...
Carregando dados de 2020 a partir de ./dados\2020\RECEITA.csv...
Carregando dados de 2021 a partir de ./dados\2021\RECEITA.csv...


In [11]:
# Pega a especificação da receita alvo
especificacao_receita_alvo = de_para[receita_alvo]
print(especificacao_receita_alvo)

['IMPOSTO SOBRE A PROPRIEDADE PREDIAL E TERRITORIAL URBANA - PRINCIPAL', {2019: '4.1.1.1.8.01.1.1.00.00.00.00.000', 2020: '4.1.1.1.8.01.1.1.00.00.00.00.000', 2021: '4.1.1.1.8.01.1.1.00.00.00.00.000'}]


In [17]:
# Busca os valores arrecadados para cada ano

# Armazena a arrecadação mensal e total da receita alvo
arrecadacao = {}
for ano in anos_anteriores:
    codigo_receita = especificacao_receita_alvo[1][ano]
    df = dados_receita[ano]
    filtro = df[df['codigo_receita'] == codigo_receita].copy()
    filtro['total'] = filtro['receita_realizada_jan'] + filtro['receita_realizada_fev'] + filtro['receita_realizada_mar'] + filtro['receita_realizada_abr'] + filtro['receita_realizada_mai'] + filtro['receita_realizada_jun'] + filtro['receita_realizada_jul'] + filtro['receita_realizada_ago'] + filtro['receita_realizada_set'] + filtro['receita_realizada_out'] + filtro['receita_realizada_nov'] + filtro['receita_realizada_dez']
    arrecadacao[ano] = filtro

In [39]:
# Calcula o percentual mensal de cada ano

# Nomes das colunas mensais
meses = colunas_nomes[1:].copy()

# Armazena os percentuais mensais de cada ano
percentuais = {}
for ano in anos_anteriores:
    percentuais[ano] = {}
    arrecadado = arrecadacao[ano]
    total = float(arrecadado['total'])
    acumulado = 0.0
    for mes in meses:
        percentuais[ano][mes] = {}
        if mes == 'receita_realizada_dez':
            percentuais[ano][mes] = total - acumulado
        else:
            percentuais[ano][mes] = float(arrecadado[mes]) / total
            acumulado += arrecadado[mes]
            
# Calcula a média mensal
percentuais_medios = {}
perc_total = 0.0
for mes in meses:
    perc_acum = 0.0
    for ano in anos_anteriores:
        perc_acum += float(percentuais[ano][mes])
    percentuais_medios[mes] = perc_acum / len(anos_anteriores)
perc_total += percentuais_medios[mes]

# Pondera os percentuais para que a sua soma corresponda a 100%

perc_acum = 0.00
for mes in meses:
    perc = percentuais_medios[mes]
    if mes == 'receita_realizada_dez':
        percentuais_medios[mes] = perc_total - perc_acum
    else:
        percentuais_medios[mes] = perc * 100 / perc_total
        perc_acum += percentuais_medios[mes]
perc_total = 1


In [41]:
# Calcula os valores mensais da receita alvo

receita_acum = 0.0
receita_alvo_sazonalizada = {}
for mes in meses:
    perc = percentuais_medios[mes]
    if mes == 'receita_realizada_dez':
        receita_alvo_sazonalizada[mes] = valor_receita_alvo - receita_acum
        receita_acum += receita_alvo_sazonalizada[mes]
    else:
        receita_alvo_sazonalizada[mes] = round(valor_receita_alvo * perc, coeficiente_arredondamento)
        receita_acum += receita_alvo_sazonalizada[mes]

In [48]:
# Mostra/salva o resultado

print(f'Janeiro:\t{receita_alvo_sazonalizada["receita_realizada_jan"]:,.2f}')
print(f'Fevereiro:\t{receita_alvo_sazonalizada["receita_realizada_fev"]:,.2f}')
print(f'Março:\t\t{receita_alvo_sazonalizada["receita_realizada_mar"]:,.2f}')
print(f'Abril:\t\t{receita_alvo_sazonalizada["receita_realizada_abr"]:,.2f}')
print(f'Maio:\t\t{receita_alvo_sazonalizada["receita_realizada_mai"]:,.2f}')
print(f'Junho:\t\t{receita_alvo_sazonalizada["receita_realizada_jun"]:,.2f}')
print(f'Julho:\t\t{receita_alvo_sazonalizada["receita_realizada_jul"]:,.2f}')
print(f'Agosto:\t\t{receita_alvo_sazonalizada["receita_realizada_ago"]:,.2f}')
print(f'Setembro:\t{receita_alvo_sazonalizada["receita_realizada_set"]:,.2f}')
print(f'Outubro:\t{receita_alvo_sazonalizada["receita_realizada_out"]:,.2f}')
print(f'Novembro:\t{receita_alvo_sazonalizada["receita_realizada_nov"]:,.2f}')
print(f'Dezembro:\t{receita_alvo_sazonalizada["receita_realizada_dez"]:,.2f}')
print(f'Total:\t{receita_acum:,.2f}')

Janeiro:	0.00
Fevereiro:	0.00
Março:		0.00
Abril:		1,000.00
Maio:		1,000.00
Junho:		3,000.00
Julho:		0.00
Agosto:		0.00
Setembro:	0.00
Outubro:	0.00
Novembro:	0.00
Dezembro:	591,000.00
Total:	5,000.00
